# Setup-Google Credentials to Download files

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.9MB/s 


#Download Embeddings and Dataset

In [0]:
#Download all Required Files using Pydrive
from numpy import genfromtxt
link='https://drive.google.com/open?id=1qo15b31PARBro7vL3K4wUAhqKaB1D8m_'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('data.csv')  

df_sub = pd.read_csv('data.csv')

#Download all Required Files using Pydrive
link='https://drive.google.com/open?id=1ygb657QqsYKYt1oVSM52qXAYb6V14sJe'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('quora_features_BERT.csv')
features = pd.read_csv('quora_features_BERT.csv')


#Embeddings3*2
link='https://drive.google.com/open?id=1OUMTHgF5CkQcIEwNfxRmRE8R4XOpYlTs'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ber_qpair_balanced.csv')
features_b = genfromtxt("ber_qpair_balanced.csv",delimiter=',',skip_header=1)


link='https://drive.google.com/open?id=18oaVUqneDcZ7Qf5zdhhazujFs4fVaENS'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('fastext_q1_balanced.csv')
ft_emb_q1  = genfromtxt("fastext_q1_balanced.csv",delimiter=',',skip_header=1)

link='https://drive.google.com/open?id=10N03ESlX0NM9UDzbrMxSTuGIEzyBZ16Z'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('fastext_q2_balanced.csv')
ft_emb_q2  = genfromtxt("fastext_q2_balanced.csv",delimiter=',',skip_header=1)


link='https://drive.google.com/open?id=13yUPFl9FDRORXCGrPpnGsTKF5Vcn-HSb'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('glove_q1_balanced.csv')
glove_emb_q1 = genfromtxt("glove_q1_balanced.csv",delimiter=',',skip_header=1)

link='https://drive.google.com/open?id=1lS0jQA7ywuEK54Q-5FAXpmPYK_LBfira'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('glove_q2_balanced.csv')
glove_emb_q2 = genfromtxt("glove_q2_balanced.csv",delimiter=',',skip_header=1)


link='https://drive.google.com/open?id=1E50i08MJXcMTV9iC7_OggCaBVWF0uq9_'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('w2vec_q1_balanced.csv')
w2v_emb_q1 = genfromtxt("w2vec_q1_balanced.csv",delimiter=',',skip_header=1)

link='https://drive.google.com/open?id=1u83-aaKQVXDIo-elNlg9vnHmo1Q63YTF'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('w2vec_q2_balanced.csv')
w2v_emb_q2 = genfromtxt("w2vec_q2_balanced.csv",delimiter=',',skip_header=1)




In [0]:
ft_emb_q1 = ft_emb_q1[:,1:]
ft_emb_q2 = ft_emb_q2[:,1:]
w2v_emb_q1 = w2v_emb_q1[:,1:]
w2v_emb_q2 = w2v_emb_q2[:,1:]
glove_emb_q1 = glove_emb_q1[:,1:]
glove_emb_q2 = glove_emb_q2[:,1:]
features = features.drop(columns = ['question1', 'question2', 'is_duplicate','jaccard_distance']).values
features_b = features_b[:,1:]

In [52]:
print('Fastext:',ft_emb_q1.shape,
ft_emb_q2.shape)

print('Word2vec:',w2v_emb_q1.shape,
w2v_emb_q2.shape)

print('Glove:',glove_emb_q1.shape,
glove_emb_q2.shape)

print('BERT features:',features.shape)

print('BERT CLS:',features_b.shape)


Fastext: (40428, 324) (40428, 324)
Word2vec: (40428, 324) (40428, 324)
Glove: (40428, 324) (40428, 324)
BERT features: (40428, 25)
BERT CLS: (40428, 768)


# Load Libraries 

In [54]:
# -*- coding: utf-8 -*-
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import time
# from bert_serving.client import BertClient

from numpy import genfromtxt
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec, FastText
# from fse.models import Sentence2Vec
# # Make sure, that the fast version of fse is available!
# from fse.models.sentence2vec import CY_ROUTINES
# assert CY_ROUTINES

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras import callbacks
import tensorflow as tf
import keras
# import tensorflow_hub as hub
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers, Add, concatenate, Layer,Lambda
from keras.optimizers import RMSprop, SGD, Adam

# Network Architecture
from keras.models import load_model

from keras.models import Sequential, Model
from keras.layers import Conv1D , MaxPooling1D, Flatten,Dense,Input,Lambda
from keras.layers import LSTM, Concatenate, Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras import backend as K
# import tensorflow as tf

import numpy as np

import torch
# from models import InferSent
from keras import regularizers

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Define Functions

In [0]:

def create_base_network_cnn(input_dimensions):

    input  = Input(shape=(input_dimensions[0],input_dimensions[1]))
    conv1  = Conv1D(filters=32,kernel_size=8,strides=1,activation = 'relu',name='conv1')(input)
    b1 = BatchNormalization()(conv1)
    d1 = Dropout(0.1)(b1)
    
    pool1  = MaxPooling1D(pool_size=1,strides=1,name='pool1')(d1)
    d2  = Dropout(0.1)(pool1)
    
    conv2  = Conv1D(filters=64,kernel_size=6,strides=1,activation = 'relu',name='conv2')(d2)
    b2 = BatchNormalization()(conv2)
    d3 = Dropout(0.1)(b2)
    
    pool2  = MaxPooling1D(pool_size=1,strides=1,name='pool2')(d3)
    d4 = Dropout(0.1)(pool2)
    
    conv3  = Conv1D(filters=128,kernel_size=4,strides=1,activation = 'relu',name='conv3')(d4)
    b3 = BatchNormalization()(conv3)
    d4 = Dropout(0.1)(b3)
    
    
    pool3  = MaxPooling1D(pool_size=1,strides=1,name='pool3')(d4)
    d5 = Dropout(0.1)(pool3)
    
    flat   = Flatten(name='flat_cnn')(d5)
    d1 = Dense(100, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(flat)
    drop1 = Dropout(0.1)(d1)
    b1 = BatchNormalization()(drop1)
    d2 = Dense(25,kernel_regularizer=regularizers.l2(0.01))(b1)
    drop2 = Dropout(0.1)(d2)
    b2 = BatchNormalization()(drop2)
    d2 = Dense(5,kernel_regularizer=regularizers.l2(0.01))(b2)
    drop3 = Dropout(0.1)(d2)
    bn = BatchNormalization()(drop3)

    model  = Model(input=input,output=bn)
  
  
    return model


def dense_network(features1):
    input = Input(shape=(features1[0],))
    #x = Flatten()(features)
    d1 = Dense(100, activation='sigmoid',kernel_regularizer=regularizers.l2(0.01))(input)
    drop1 = Dropout(0.1)(d1)
    b1 = BatchNormalization()(drop1)
    d2 = Dense(25,kernel_regularizer=regularizers.l2(0.01))(b1)
    drop2 = Dropout(0.1)(d2)
    b2 = BatchNormalization()(drop2)
    d2 = Dense(5,kernel_regularizer=regularizers.l2(0.01))(b2)
    drop3 = Dropout(0.1)(d2)
    b3 = BatchNormalization()(drop3)
#     flat   = Flatten(name='flat_dnn2')(b3)
    model = Model(input = input,output=b3)
    return model
  

def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)



def create_network(input_dimensions,num_features):

#     # #Fasttext
    base_network_lstm_1 = dense_network([324,1])
    input_a_lstm_1 = Input(shape=(input_dimensions[0],))
    input_b_lstm_1 = Input(shape=(input_dimensions[0],))
    # LSTM with embedding 1
    inter_a_lstm_1 = base_network_lstm_1(input_a_lstm_1)
    inter_b_lstm_1 = base_network_lstm_1(input_b_lstm_1)
    d_lstm_1 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_1, inter_b_lstm_1])


    #W2V
    base_network_lstm_2 = dense_network([324,1])
    input_a_lstm_2 = Input(shape=(input_dimensions[0],))
    input_b_lstm_2 = Input(shape=(input_dimensions[0],))
    # LSTM with embedding 2
    inter_a_lstm_2 = base_network_lstm_2(input_a_lstm_2)
    inter_b_lstm_2 = base_network_lstm_2(input_b_lstm_2)
    d_lstm_2 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_2, inter_b_lstm_2])


    #Glove
    base_network_lstm_3 = dense_network([324,1])
    input_a_lstm_3 = Input(shape=(input_dimensions[0],))
    input_b_lstm_3 = Input(shape=(input_dimensions[0],))
    # LSTM with embedding 3
    inter_a_lstm_3 = base_network_lstm_3(input_a_lstm_3)
    inter_b_lstm_3 = base_network_lstm_3(input_b_lstm_3)
    d_lstm_3 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_3, inter_b_lstm_3])

    #Uncomment to Use BERT as Siamese as well
#     base_network_lstm_4 = dense_network([768,1])
#     input_a_lstm_4 = Input(shape=(768,))
#     input_b_lstm_4 = Input(shape=(768,))
#     # LSTM with embedding 3
#     inter_a_lstm_4 = base_network_lstm_4(input_a_lstm_4)
#     inter_b_lstm_4 = base_network_lstm_4(input_b_lstm_4)


#     CNN
    base_network_cnn = create_base_network_cnn(input_dimensions)
    # CNN with 3 channel embedding
    input_a_cnn = Input(shape=(input_dimensions[0],input_dimensions[1]))
    input_b_cnn = Input(shape=(input_dimensions[0],input_dimensions[1]))
    inter_a_cnn = base_network_cnn(input_a_cnn)
    inter_b_cnn = base_network_cnn(input_b_cnn)



    d_cnn = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_cnn, inter_b_cnn])
    d_lstm_1 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_1, inter_b_lstm_1])
    d_lstm_2 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_2, inter_b_lstm_2])
    d_lstm_3 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_3, inter_b_lstm_3])
#     d_lstm_4 = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([inter_a_lstm_4, inter_b_lstm_4])

    # Additional Features from (BERT)
    features = Input(shape=(num_features,))

    #BERT itself
    features_b = Input(shape=(768,))


    #Concatenation of Features
    feature_set = Concatenate(axis=-1)([d_cnn,d_lstm_1,d_lstm_2,d_lstm_3,features,features_b])




    #x = Flatten()(features)
    d1 = Dense(300, activation='relu',kernel_regularizer=regularizers.l2(0.01))(feature_set)
    drop1 = Dropout(0.1)(d1)
    b1 = BatchNormalization()(drop1)
    d2 = Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001))(b1)
    drop2 = Dropout(0.1)(d2)
    b2 = BatchNormalization()(drop2)
    d3 = Dense(2, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(b2)


#     model = Model(input=[feature_set,features_b], output=d3)
    model = Model(input=[input_a_cnn, input_b_cnn , input_a_lstm_1, input_b_lstm_1, input_a_lstm_2, input_b_lstm_2, input_a_lstm_3, input_b_lstm_3,features,features_b], output=d3)  
    print("Model Architecture Designed")
    return model
  


# Load Data and Embeddings from Downloaded files

In [0]:
df_sub['question1'] = df_sub['question1'].apply(lambda x: str(x))
df_sub['question2'] = df_sub['question2'].apply(lambda x: str(x))
q1sents = list(df_sub['question1'])
q2sents = list(df_sub['question2'])
tokenized_q1sents = [word_tokenize(i) for i in list(df_sub['question1'])]
tokenized_q2sents = [word_tokenize(i) for i in list(df_sub['question2'])]

# Prepare-Train-Test-Validation Data for Model

In [60]:
num_train = int(df_sub.shape[0] * 0.70)
num_val = int(df_sub.shape[0] * 0.10)
num_test = df_sub.shape[0] - num_train - num_val 

print("Number of training pairs: %i"%(num_train))
print("Number of Validation pairs: %i"%(num_val))
print("Number of testing pairs: %i"%(num_test))

Number of training pairs: 28299
Number of Validation pairs: 4042
Number of testing pairs: 8087


In [0]:
# init data data arrays
X_train_cnn_a = np.zeros([num_train, 324, 3])
X_test_cnn_a  = np.zeros([num_test, 324, 3])
X_val_cnn_a  = np.zeros([num_val, 324, 3])

X_train_cnn_b = np.zeros([num_train, 324, 3])
X_test_cnn_b  = np.zeros([num_test, 324, 3])
X_val_cnn_b  = np.zeros([num_val, 324, 3])

Y_train = np.zeros([num_train]) 
Y_test = np.zeros([num_test])
Y_val = np.zeros([num_val]) 


#Labels
Y_train = df_sub['is_duplicate'].values[num_train]
Y_val = df_sub['is_duplicate'].values[num_val]
Y_test = df_sub['is_duplicate'].values[num_val]



num_val = num_train + int(df_sub.shape[0] * 0.10)
# fill data arrays with features
X_train_cnn_a[:,:,0] = ft_emb_q1[:num_train]
X_train_cnn_a[:,:,1] = w2v_emb_q1[:num_train]
X_train_cnn_a[:,:,2] = glove_emb_q1[:num_train]
# X_train_cnn_a[:,:,3] = bert_q1_pca[:num_train]

X_train_cnn_b[:,:,0] = ft_emb_q2[:num_train]
X_train_cnn_b[:,:,1] = w2v_emb_q2[:num_train]
X_train_cnn_b[:,:,2] = glove_emb_q2[:num_train]
# X_train_cnn_b[:,:,3] = bert_q2_pca[:num_train]

features_train = features[:num_train]
features_b_train = features_b[:num_train]
Y_train = df_sub[:num_train]['is_duplicate'].values

X_val_cnn_a[:,:,0] = ft_emb_q1[num_train:num_val]
X_val_cnn_a[:,:,1] = w2v_emb_q1[num_train:num_val]
X_val_cnn_a[:,:,2] = glove_emb_q1[num_train:num_val]
# X_val_cnn_a[:,:,3] = bert_q1_pca[num_train:num_val]

X_val_cnn_b[:,:,0] = ft_emb_q2[num_train:num_val]
X_val_cnn_b[:,:,1] = w2v_emb_q2[num_train:num_val]
X_val_cnn_b[:,:,2] = glove_emb_q2[num_train:num_val]
# X_val_cnn_b[:,:,3] = bert_q2_pca[num_train:num_val]

features_val = features[num_train:num_val]
features_b_val =features_b[num_train:num_val]
Y_val = df_sub[num_train:num_val]['is_duplicate'].values


X_test_cnn_a[:,:,0] = ft_emb_q1[num_val:]
X_test_cnn_a[:,:,1] = w2v_emb_q1[num_val:]
X_test_cnn_a[:,:,2] = glove_emb_q1[num_val:]
# X_test_cnn_a[:,:,3] = bert_q1_pca[num_val:]

X_test_cnn_b[:,:,0] = ft_emb_q2[num_val:]
X_test_cnn_b[:,:,1] = w2v_emb_q2[num_val:]
X_test_cnn_b[:,:,2] = glove_emb_q2[num_val:]
# X_test_cnn_b[:,:,3] = bert_q2_pca[num_val:]


features_test = features[num_val:]
features_b_test = features_b[num_val:]
Y_test = df_sub[num_val:]['is_duplicate'].values


Y_train_old = Y_train
Y_test_old = Y_test
Y_val_old = Y_val

Y_train = keras.utils.to_categorical(Y_train, num_classes=2)
Y_test = keras.utils.to_categorical(Y_test, num_classes=2)
Y_val = keras.utils.to_categorical(Y_val, num_classes=2)

In [64]:
# # Training Sets for Siamese1
X_intera_train_1 = X_train_cnn_a[:,:,0]
X_interb_train_1 = X_train_cnn_b[:,:,0]
X_train_lstm1_a = X_intera_train_1
X_train_lstm1_b = X_interb_train_1

X_intera_val_1 = X_val_cnn_a[:,:,0]
X_interb_val_1 = X_val_cnn_b[:,:,0]
X_val_lstm1_a = X_intera_val_1
X_val_lstm1_b = X_interb_val_1

X_intera_test_1 = X_test_cnn_a[:,:,0]
X_interb_test_1 = X_test_cnn_b[:,:,0]
X_test_lstm1_a = X_intera_test_1
X_test_lstm1_b = X_interb_test_1


# Sets for Siamese2
X_intera_train_2 = X_train_cnn_a[:,:,1]
X_interb_train_2 = X_train_cnn_b[:,:,1]
X_train_lstm2_a = X_intera_train_2
X_train_lstm2_b = X_interb_train_2

X_intera_val_2 = X_val_cnn_a[:,:,1]
X_interb_val_2 = X_val_cnn_b[:,:,1]
X_val_lstm2_a = X_intera_val_2
X_val_lstm2_b = X_interb_val_2

X_intera_test_2 = X_test_cnn_a[:,:,1]
X_interb_test_2 = X_test_cnn_b[:,:,1]
X_test_lstm2_a = X_intera_test_2
X_test_lstm2_b = X_interb_test_2

# Set for Siamese3

X_intera_train_3 = X_train_cnn_a[:,:,2]
X_interb_train_3 = X_train_cnn_b[:,:,2]
X_train_lstm3_a = X_intera_train_3
X_train_lstm3_b = X_interb_train_3

X_intera_val_3 = X_val_cnn_a[:,:,2]
X_interb_val_3 = X_val_cnn_b[:,:,2]
X_val_lstm3_a = X_intera_val_3
X_val_lstm3_b = X_interb_val_3
X_intera_test_3 = X_test_cnn_a[:,:,2]
X_interb_test_3 = X_test_cnn_b[:,:,2]
X_test_lstm3_a = X_intera_test_3
X_test_lstm3_b = X_interb_test_3


# Test Set for LSTM4 BERT

# X_intera_train_4 = bert_q1[:num_train]
# X_train_lstm4_a = X_intera_train_4

# X_intera_val_4 = bert_q1[num_train:num_val]
# X_val_lstm4_a = X_intera_val_4

# X_intera_test_4 = bert_q1[num_val:]
# X_test_lstm4_a = X_intera_test_4

# X_interb_train_4 = bert_q2[:num_train]
# X_train_lstm4_b = X_interb_train_4

# X_interb_val_4 = bert_q2[num_train:num_val]
# X_val_lstm4_b = X_interb_val_4

# X_interb_test_4 = bert_q2[num_val:]
# X_test_lstm4_b = X_interb_test_4
# X_test_lstm4_b = X_interb_test_4

print("Input Shapes")
print("CNN Shape")
print(X_train_cnn_a.shape,X_val_cnn_a.shape,X_test_cnn_a.shape)
print("LSTM (x3) Shape:")
print(X_train_lstm1_a.shape,X_val_lstm1_a.shape,X_test_lstm1_a.shape)
# print("LSTM (BERT) Shape:")
# print(X_train_lstm4_a.shape,X_val_lstm4_a.shape,X_test_lstm4_a.shape)

print("Features shape:",features_train.shape,features_val.shape,features_test.shape)
print("BERT Features shape:",features_b_train.shape,features_b_val.shape,features_b_test.shape)

print("Labels Shape")
print(Y_train.shape,Y_val.shape,Y_test.shape)



Input Shapes
CNN Shape
(28299, 324, 3) (4042, 324, 3) (8087, 324, 3)
LSTM (x3) Shape:
(28299, 324) (4042, 324) (8087, 324)
Features shape: (28299, 25) (4042, 25) (8087, 25)
BERT Features shape: (28299, 768) (4042, 768) (8087, 768)
Labels Shape
(28299, 2) (4042, 2) (8087, 2)


# Training with DNN as Classification Layer

In [69]:
net = create_network([324,3],25)
optimizer = Adam(lr=0.001)
net.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=['accuracy'])
print(net.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
W0807 07:59:20.282669 140614001997696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ba...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:151: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
W0807 07:59:23.125292 140614001997696 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0807 07:59:23

Model Architecture Designed
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 324, 3)       0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 324, 3)       0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 324)          0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 324)          0                                            
_________________________________________________________________________________

In [0]:
filepath="./Model7_{epoch:02d}_{val_acc:.4f}.h5"
checkpoint = callbacks.ModelCheckpoint(filepath, 
                                    monitor='val_acc', 
                                    verbose=0, 
                                    save_best_only=True)
callbacks_list = [checkpoint]


net.fit([ X_train_cnn_a, X_train_cnn_b,X_train_lstm1_a, X_train_lstm1_b,
            X_train_lstm2_a, X_train_lstm2_b,X_train_lstm3_a, X_train_lstm3_b,features_train,features_b_train], 
            Y_train,
          validation_data=([X_val_cnn_a, X_val_cnn_b,X_val_lstm1_a, X_val_lstm1_b,
                          X_val_lstm2_a, X_val_lstm2_b,X_val_lstm3_a, X_val_lstm3_b,features_val,features_b_val]
                          , Y_val),
          batch_size=384, nb_epoch=20, shuffle=True,callbacks = callbacks_list)

score = net.evaluate([X_test_cnn_a, X_test_cnn_b,X_test_lstm1_a, X_test_lstm1_b,
              X_test_lstm2_a, X_test_lstm2_b,X_test_lstm3_a, X_test_lstm3_b,features_test,features_b_test],Y_test,batch_size=384)

print('Test loss : {:.4f}'.format(score[0]))
print('Test accuracy : {:.4f}'.format(score[1]))


# Loading the Best Saved Model

In [73]:
#Download all Required Files using Pydrive
link='https://drive.google.com/open?id=1mnDiANiu_m16IdWZdGewKKUiqGkFSjbx'
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Model1_40_0.7984.h5')
model = load_model('Model1_40_0.7984.h5')
model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=['acc'])

score = model.evaluate([X_test_cnn_a, X_test_cnn_b,X_test_lstm1_a, X_test_lstm1_b,
              X_test_lstm2_a, X_test_lstm2_b,X_test_lstm3_a, X_test_lstm3_b,features_test,features_b_test],Y_test,batch_size=128)
print('Test loss : {:.4f}'.format(score[0]))
print('Test accuracy : {:.4f}'.format(score[1]))


8087/8087 [==============================] - 29s 4ms/step
Test loss : 0.5678
Test accuracy : 0.8018


# Classification with SVM

In [0]:

# Getting features for classical machine learning algorithms
def feature_extractor():
  intermediate_layer_model = Model(inputs=net.input,
                                 outputs=net.layers[-8].output)
  data_train = [ X_train_cnn_a, X_train_cnn_b,X_train_lstm1_a, X_train_lstm1_b,
            X_train_lstm2_a, X_train_lstm2_b,X_train_lstm3_a, X_train_lstm3_b,features_train,features_b_train]

  data_validation = [X_val_cnn_a, X_val_cnn_b,X_val_lstm1_a, X_val_lstm1_b,
                          X_val_lstm2_a, X_val_lstm2_b,X_val_lstm3_a, X_val_lstm3_b,features_val,features_b_val]

  data_test = [X_test_cnn_a, X_test_cnn_b,X_test_lstm1_a, X_test_lstm1_b,
              X_test_lstm2_a, X_test_lstm2_b,X_test_lstm3_a, X_test_lstm3_b,features_test,features_b_test]


  intermediate_output_train = intermediate_layer_model.predict(data_train)
  intermediate_output_validation = intermediate_layer_model.predict(data_validation)
  intermediate_output_test = intermediate_layer_model.predict(data_test)
  
  features_combined = [intermediate_output_train, intermediate_output_validation,intermediate_output_test]
  return features_combined

[feats_train,feats_val,feats_test] = feature_extractor()

# Getting data for classical machine learning algorithms


train_feats = np.array(list(feats_train) + list(feats_val))
train_Y     = np.array(list(Y_train_old) + list(Y_val_old))

test_feats = feats_test
test_Y = Y_test_old

print("train_feats :",train_feats.shape)
print("train_Y : ",train_Y.shape)

print("test_feats : ",test_feats.shape)
print("test_Y : ",test_Y.shape)

# Support Vector Machine
from sklearn.svm import SVC
svm_clf = SVC(gamma='auto')
svm_clf.fit(train_feats,train_Y)
print(svm_clf.score(test_feats,test_Y))

# Classification with XGBOOST

In [0]:
# XGBoost
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

model_xg = XGBClassifier()
model_xg.fit(train_feats, train_Y)
y_pred_xg = model.predict(test_feats)
predictions_xg = [round(value) for value in y_pred_xg]
accuracy = accuracy_score(test_feats, predictions_xg)

print(accuracy)